In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 2 * matplotlib.rcParams['savefig.dpi']

In [2]:
# data
# http://data.beta.nyc/dataset/unofficial-mta-transit-data-archive

# documentation 
# http://bustime.mta.info/wiki/Developers/ArchiveData

#gtfs spec
# https://developers.google.com/transit/gtfs/reference
!wget -nc http://data.mytransit.nyc.s3.amazonaws.com/bus_time/2015/2015-01/bus_time_20150128.csv.xz
!7z -y x bus_time_20150128.csv.xz
!wget -nc http://data.mytransit.nyc.s3.amazonaws.com/bus_time/2015/2015-01/bus_time_20150129.csv.xz
!7z -y x bus_time_20150129.csv.xz

!wget -nc http://data.mytransit.nyc.s3.amazonaws.com/gtfs/2015/gtfs_nyct_bus_20150103.zip
!unzip -o gtfs_nyct_bus_20150103.zip

--2017-11-21 13:43:23--  http://data.mytransit.nyc.s3.amazonaws.com/bus_time/2015/2015-01/bus_time_20150128.csv.xz
Resolving data.mytransit.nyc.s3.amazonaws.com (data.mytransit.nyc.s3.amazonaws.com)... 52.216.229.251
Connecting to data.mytransit.nyc.s3.amazonaws.com (data.mytransit.nyc.s3.amazonaws.com)|52.216.229.251|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41847772 (40M) [application/x-xz]
Saving to: ‘bus_time_20150128.csv.xz’

100%[======================================>] 41,847,772  64.8MB/s   in 0.6s   

2017-11-21 13:43:24 (64.8 MB/s) - ‘bus_time_20150128.csv.xz’ saved [41847772/41847772]


7-Zip [64] 9.20  Copyright (c) 1999-2010 Igor Pavlov  2010-11-18
p7zip Version 9.20 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,8 CPUs)

Processing archive: bus_time_20150128.csv.xz

Extracting  bus_time_20150128.csv

Everything is Ok

Size:       359163860
Compressed: 41847772
--2017-11-21 13:43:31--  http://data.mytransit.nyc.s3.amazonaws.com/bus_time/2015/2015-

In [14]:
#ETL
import pandas as pd
import numpy as np
from datetime import timedelta, datetime


def convert_to_int64(row):
    try:
        return np.int64(row)
    except ValueError:
        return np.nan

def utc_to_est(row):
    try:
        return row - timedelta(hours=5)
    except:
        raise

csv_f = pd.read_csv
archive = pd.concat([csv_f("bus_time_20150128.csv"),
                     csv_f("bus_time_20150129.csv")])

trips = pd.read_csv("trips.txt")
stops = pd.read_csv("stops.txt")
schedules = pd.read_csv("stop_times.txt")
#schedules.departure_time = pd.to_datetime(schedules.departure_time.apply(_25_to_other))
#archive.tim1estamp = pd.to_datetime(archive.timestamp).apply(utc_to_est)
archive.next_stop_id = archive.next_stop_id.apply(convert_to_int64)

In [12]:
def _25_to_other(row):
    r = int(row.split(":")[0])
    if r > 23:
        r = "0" + str(r - 24)
        return ":".join([r] + row.split(":")[1:])
    else:
        return row
    
schedules.departure_time.apply(_25_to_other)

0          00:15:00
1          00:15:55
2          00:17:07
3          00:17:37
4          00:18:05
5          00:18:42
6          00:19:47
7          00:20:14
8          00:21:00
9          00:21:59
10         00:22:26
11         00:23:28
12         00:24:12
13         00:25:00
14         00:25:27
15         00:25:53
16         00:26:13
17         00:27:01
18         00:27:23
19         00:27:47
20         00:28:13
21         00:28:34
22         00:29:00
23         00:29:27
24         00:30:14
25         00:31:06
26         00:31:54
27         00:33:00
28         00:34:14
29         00:35:00
             ...   
8855813    02:49:00
8855814    02:49:26
8855815    02:49:58
8855816    02:50:14
8855817    02:50:38
8855818    02:51:06
8855819    02:51:26
8855820    02:51:45
8855821    02:52:04
8855822    02:53:00
8855823    02:53:21
8855824    02:54:27
8855825    02:55:06
8855826    02:56:00
8855827    02:56:31
8855828    02:57:04
8855829    02:57:50
8855830    02:58:05
8855831    02:58:35


In [15]:
#query and clean
live_archive = archive[archive.block_assigned > 0] # "assigned" to a route
today = live_archive[live_archive.service_date == 20150128] # due to time shift we get some dates in 1/27 and 1/29
#throw away trips with <15 reports
good_trips_only = today.groupby(today.trip_id).filter(lambda group: len(group) > 15)

#write this as our "clean" dataset
good_trips_only.to_csv("realtime.csv")

#merged dataframe with both 
partial = good_trips_only.merge(trips, on='trip_id') 
df = partial.merge(stops, left_on="next_stop_id", right_on="stop_id")

In [21]:
pd.options.display.max_columns = None
m15_only = df[df.route_id.str.contains("M15")]


array(['M15', 'M15+'], dtype=object)

In [23]:
m15_only.head()

,timestamp,vehicle_id,latitude,longitude,bearing,progress,service_date,trip_id,block_assigned,next_stop_id,dist_along_route,dist_from_stop,route_id,service_id,trip_headsign,direction_id,shape_id,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
161849,2015-01-28T05:23:55Z,6786,40.803310,-73.933166,233.93,0,20150128,OH_A5-Weekday-SDon-002000_M15_2,1,401738,180.23,108.89,M15,OH_A5-Weekday-SDon,SOUTH FERRY via 2 AV,1,M150351,401738,2 AV/E 125 ST,NaN,40.802494,-73.933891,NaN,NaN,0,NaN
161850,2015-01-28T05:24:26Z,6786,40.802600,-73.933682,234.02,0,20150128,OH_A5-Weekday-SDon-002000_M15_2,1,401738,180.23,18.77,M15,OH_A5-Weekday-SDon,SOUTH FERRY via 2 AV,1,M150351,401738,2 AV/E 125 ST,NaN,40.802494,-73.933891,NaN,NaN,0,NaN
161851,2015-01-28T06:07:21Z,3904,40.803394,-73.933011,157.83,0,20150128,OH_A5-Weekday-SDon-006000_M15_3,1,401738,180.23,130.16,M15,OH_A5-Weekday-SDon,SOUTH FERRY via 2 AV,1,M150351,401738,2 AV/E 125 ST,NaN,40.802494,-73.933891,NaN,NaN,0,NaN
161852,2015-01-28T06:07:53Z,3904,40.802610,-73.933675,234.02,0,20150128,OH_A5-Weekday-SDon-006000_M15_3,1,401738,180.23,20.03,M15,OH_A5-Weekday-SDon,SOUTH FERRY via 2 AV,1,M150351,401738,2 AV/E 125 ST,NaN,40.802494,-73.933891,NaN,NaN,0,NaN
161853,2015-01-28T06:33:26Z,3892,40.802570,-73.933704,234.02,0,20150128,OH_A5-Weekday-SDon-009000_M15_4,1,401738,180.23,15.02,M15,OH_A5-Weekday-SDon,SOUTH FERRY via 2 AV,1,M150351,401738,2 AV/E 125 ST,NaN,40.802494,-73.933891,NaN,NaN,0,NaN
